In [1]:
import rdflib
import json

# Load the ontology
g = rdflib.Graph()
g.parse("rdf/draft-topic-onto.rdf")

# Define start and end topics
startID = "Matrices and linear algebra fundamentals"
endID = "Machine Learning"

# List to store the paths
paths = []

# Define the DFS function
def DFS(stack, endID, path, paths):
    if not stack:
        DFS([startID] + stack, endID, path + [startID], paths)
    else:
        if path != [] and path[-1] == endID:
            paths += [path]
            return
        
        # Create a SPARQL query to find the next topics
        sparql_query = f"""
        PREFIX : <http://www.semanticweb.org/thuha/topic-onto/>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xml: <http://www.w3.org/XML/1998/namespace>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX topics: <http://www.semanticweb.org/thuha/topic-onto#>
        BASE <http://www.semanticweb.org/thuha/topic-onto/>
        SELECT *
        WHERE {{
            ?start topics:topicID "{stack[0]}".
            ?start topics:link ?topicID.
        }}
        """

        # Execute the SPARQL query
        qres = g.query(sparql_query)
        for row in qres:
            DFS([row["topicID"].value] + stack, endID, path + [row["topicID"].value], paths)

# Execute the DFS function to find paths
DFS([], endID, [], paths)

# Print the found paths
for path in paths:
    print(path)

# Save the paths to a JSON file
with open('paths.json', 'w') as json_file:
    json.dump(paths, json_file, indent=2)


['Matrices and linear algebra fundamentals', 'K-Means Clustering', 'DBSCAN', 'HDBSCAN', 'Machine Learning']
['Matrices and linear algebra fundamentals', 'Data mining', 'Machine Learning']


In [24]:
import rdflib
import json

# Function to get learning materials based on topicID
def getLMtopic(topicID):
    sparql_query = f"""
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX onto: <http://www.semanticweb.org/thuha/ontologies/system-ontology#>
    BASE <http://www.semanticweb.org/thuha/ontologies/system-ontology/>
    SELECT *
    WHERE {{
        ?lm onto:topic "{topicID}".
        ?lm onto:lmID ?lmID.
        ?lm onto:material_ratings ?rating.
        ?lm onto:score ?maxScore.
        ?lm onto:time ?maxTime.
        ?lm onto:difficulty ?difficulty.
    }}
    """
    qres = g.query(sparql_query)

    lms = []
    for row in qres:
        #print(row["rating"].value,row["lmID"].value)
        lms.append((row["rating"].value, 0, row["lmID"].value))

    return lms

# Load the ontology
g = rdflib.Graph()
g.parse("rdf/draft-system-onto.rdf")

qualification = "Graduate"
backgroundKnowledge = "Basic"
active_reflective = "0"
visual_verbal = "1"
global_sequential = "1"
sensitive_intuitive = "0"

sparql_query = f"""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX onto: <http://www.semanticweb.org/thuha/ontologies/system-ontology#>
BASE <http://www.semanticweb.org/thuha/ontologies/system-ontology/>
SELECT *
WHERE {{
    ?learner onto:qualification "{qualification}".
    ?learner onto:backgroundKnowledge "{backgroundKnowledge}".
    ?learner onto:active_reflective "{active_reflective}"^^xsd:decimal.
    ?learner onto:visual_verbal "{visual_verbal}"^^xsd:decimal.
    ?learner onto:global_sequential "{global_sequential}"^^xsd:decimal.
    ?learner onto:sensitive_intuitive "{sensitive_intuitive}"^^xsd:decimal.
    ?learner onto:learnerID ?learnerID.
    ?log onto:learnerID ?learnerID.
    ?log onto:lmID ?lmID.
    ?log onto:attempt ?attempt.
    ?log onto:score ?score.
    ?log onto:time ?time.
    ?lm onto:lmID ?lmID.
    ?lm onto:material_ratings ?rating.
    ?lm onto:topic ?topicID.
    ?lm onto:score ?maxScore.
    ?lm onto:time ?maxTime.
    ?lm onto:difficulty ?difficulty.
}}
"""
qres = g.query(sparql_query)

# with open('json/paths.json', 'r') as json_file:
#     paths = json.load(json_file)

paths = [
    ["Matrices and linear algebra fundamentals","K-Means Clustering","DBSCAN","HDBSCAN","Machine Learning"],
    ["Matrices and linear algebra fundamentals","Data mining","Machine Learning"]
    ]
for path in paths:
    lms = {}
    topics = {}
    for topic in path:
        topics[topic] = []
    for row in qres:
        #print('row_id: ',row["topicID"].value)
        if row["topicID"].value not in topics:
            continue

        topicID = row["topicID"].value
        lmID = row["lmID"].value
        rating = int(row["rating"].value)
        #print(topicID, lmID, rating)
        if lmID not in lms:
            lms[lmID] = {
                "score": float(row["score"].value),
                "time": float(row["time"].value),
                "attempt": int(row["attempt"].value),
                "difficulty": float(row["difficulty"].value),
                "rating": rating,
                "topic": topicID,
                "maxScore": float(row["maxScore"].value),
                "maxTime": float(row["maxTime"].value),
                "repeat": 1
            }
        else:
            lms[lmID]["score"] += float(row["score"].value)
            lms[lmID]["time"] += float(row["time"].value)
            lms[lmID]["attempt"] += int(row["attempt"].value)
            lms[lmID]["repeat"] += 1
    #print(lms)
    for lm in lms:
        aScore = lms[lm]["score"] / lms[lm]["repeat"]
        aTime = lms[lm]["time"] / lms[lm]["repeat"]
        aAttempt = lms[lm]["attempt"] / lms[lm]["repeat"]

        similarity = abs(0.4 * (aScore / lms[lm]["maxScore"]) + 0.3 * (1.0 - aTime / lms[lm]["maxTime"]) + 0.3 * (1.0 - aAttempt) - lms[lm]["difficulty"])
        topics[lms[lm]["topic"]].append((lms[lm]["rating"], similarity, lm))
    print(topics)
    for topic in topics:
        if topics[topic] == []:
            topics[topic] = getLMtopic(topic)
        topics[topic].sort(key=lambda x: x[0], reverse=True)
        end = round(len(topics[topic]) * 0.2 + 0.5)
        topics[topic] = topics[topic][:end]
        topics[topic].sort(key=lambda x: x[1])

    for topic in topics:
        
        #print(topic)
        print(str(topic) + " - " + topics[topic][0][2])

    print("\n================================")


{'Matrices and linear algebra fundamentals': [(4, 0.7799999999999998, 'Document C')], 'K-Means Clustering': [], 'DBSCAN': [(3, 1.1599999999999997, 'Video B'), (5, 0.9799999999999999, 'Video A')], 'HDBSCAN': [], 'Machine Learning': [(2, 0.7799999999999998, 'Code C')]}
Matrices and linear algebra fundamentals - Document C
K-Means Clustering - 2
DBSCAN - Video A
HDBSCAN - Video C
Machine Learning - Code C

{'Matrices and linear algebra fundamentals': [(4, 0.7799999999999998, 'Document C')], 'Data mining': [], 'Machine Learning': [(2, 0.7799999999999998, 'Code C')]}
Matrices and linear algebra fundamentals - Document C
Data mining - Document D
Machine Learning - Code C



In [20]:
import rdflib
import json

class SpraqlTopic:
    def __init__(self, startID, endID, path):
        self.g = rdflib.Graph()
        self.g.parse(path)
        self.startID = startID
        self.endID = endID
    
    def convertInt(self, value):
        try:
            return int(value)
        except ValueError:
            return value
        
    def DFS(self, stack, path, paths):
        if not stack:
            self.DFS([self.startID] + stack, path + [self.startID], paths)
        else:
            if path != [] and path[-1] == self.endID:
                paths.append(path)  # Store path with string IDs
                return
            
            sparql_query = f"""
                PREFIX : <http://www.semanticweb.org/thuha/topic-onto/>
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX xml: <http://www.w3.org/XML/1998/namespace>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX topics: <http://www.semanticweb.org/thuha/topic-onto#>
                BASE <http://www.semanticweb.org/thuha/topic-onto/>
                SELECT ?topicID
                WHERE
                {{
                    ?start topics:topicID "{stack[0]}".
                    ?start topics:link ?topicID.
                }}
            """
            
            qres = self.g.query(sparql_query)
            for row in qres:
                topic_id = row["topicID"].value
                converted_id = self.convertInt(topic_id)
                if topic_id != 'None' and topic_id not in path:
                    self.DFS([topic_id] + stack, path + [converted_id], paths)

    def spraqlTopic(self):
        paths = []
        self.DFS([], [], paths)
        with open('paths.json', 'w') as json_file:
            json.dump(paths, json_file, indent=2)
        return paths

# Usage
startID = "Matrices and linear algebra fundamentals"
endID = "Machine Learning"
rdf_path = "rdf/draft-topic-onto.rdf"

spraql_topic = SpraqlTopic(startID, endID, rdf_path)
paths = spraql_topic.spraqlTopic()

# Print the paths
for path in paths:
    print(",".join(map(str, path)))


Matrices and linear algebra fundamentals,K-Means Clustering,DBSCAN,HDBSCAN,Machine Learning
Matrices and linear algebra fundamentals,Data mining,Machine Learning


In [1]:
import requests